In [17]:
#run in python 3.10.7 on windows

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

auckland_rain  = pd.read_csv('../data/niwa_cleaned/niwaDailyWeatherData.csv', parse_dates=['Date(NZST)'])
auckland_rain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 945 entries, 0 to 944
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Unnamed: 0            945 non-null    int64         
 1   Date(NZST)            945 non-null    datetime64[ns]
 2   Pmsl(hPa)             945 non-null    float64       
 3   Pstn(hPa)             945 non-null    float64       
 4   Amount(mm)            941 non-null    float64       
 5   Period(min)           945 non-null    float64       
 6   daily_total_rainfall  945 non-null    float64       
 7   Rainfall(mm)          945 non-null    float64       
 8   Deficit(mm)           945 non-null    float64       
 9   Sunshine(Hrs)         945 non-null    float64       
 10  WindDir(DegT)         945 non-null    int64         
 11  Speed(m/s)_x          945 non-null    float64       
 12  WindDir StdDev        945 non-null    int64         
 13  WindSpd StdDev      

In [18]:
auckland_rain.head()

,Unnamed: 0,Date(NZST),Pmsl(hPa),Pstn(hPa),Amount(mm),Period(min),daily_total_rainfall,Rainfall(mm),Deficit(mm),Sunshine(Hrs),...,Tgmin(C),Tmean(C),Twet(C),RH(%),Tdew(C),Dir(DegT),Speed(m/s)_y,Percent(%),Depth(cm),soil_avg_moist
0,0,2019-01-01,1016.5,993.2,0.0,4878.8,0.75,0.0,40.7,5.3,...,12.5,20.0,16.2,82.0,14.9,188,6.2,40.9,20,40.9
1,1,2019-01-02,1013.6,990.5,0.0,534.9,0.15,0.6,44.6,2.2,...,15.8,21.8,17.7,82.0,16.5,13,6.2,40.4,20,40.4
2,2,2019-01-03,1011.3,988.2,0.0,1416.8,0.10,0.0,49.2,8.6,...,13.2,20.4,17.2,82.0,16.0,201,8.8,40.5,20,40.5
3,3,2019-01-06,1016.2,992.9,0.0,4621.0,1.32,0.0,62.8,7.4,...,14.5,20.1,17.4,88.0,16.6,353,9.8,37.2,20,37.2
4,4,2019-01-07,1019.2,995.6,0.0,56.8,0.28,1.6,65.7,10.3,...,13.3,20.5,12.9,68.0,10.1,162,9.3,36.6,20,36.6


In [19]:
# Summary Statistics
print(auckland_rain.describe())

       Unnamed: 0                     Date(NZST)    Pmsl(hPa)    Pstn(hPa)  \
count  945.000000                            945   945.000000   945.000000   
mean   472.000000  2021-04-23 13:33:42.857143040  1014.103386   990.160635   
min      0.000000            2019-01-01 00:00:00   978.100000   954.800000   
25%    236.000000            2020-02-29 00:00:00  1007.900000   984.300000   
50%    472.000000            2021-05-08 00:00:00  1014.500000   990.700000   
75%    708.000000            2022-06-24 00:00:00  1020.700000   996.800000   
max    944.000000            2023-08-19 00:00:00  1041.500000  1016.200000   
std    272.942302                            NaN     9.725952     9.440421   

       Amount(mm)   Period(min)  daily_total_rainfall  Rainfall(mm)  \
count  941.000000    945.000000            945.000000    945.000000   
mean    -0.270138   1912.096296              6.592095      8.774815   
min    -30.000000      1.400000            -33.000000      0.000000   
25%      0.00

In [20]:
# Handling Missing Data
missing_data = auckland_rain.isnull().sum()
print(missing_data)

Unnamed: 0              0
Date(NZST)              0
Pmsl(hPa)               0
Pstn(hPa)               0
Amount(mm)              4
Period(min)             0
daily_total_rainfall    0
Rainfall(mm)            0
Deficit(mm)             0
Sunshine(Hrs)           0
WindDir(DegT)           0
Speed(m/s)_x            0
WindDir StdDev          0
WindSpd StdDev          0
Tmax(C)                 0
Tmin(C)                 0
Tgmin(C)                0
Tmean(C)                0
Twet(C)                 0
RH(%)                   0
Tdew(C)                 0
Dir(DegT)               0
Speed(m/s)_y            0
Percent(%)              0
Depth(cm)               0
soil_avg_moist          0
dtype: int64


In [21]:
# Create lag features for prevous 7 days of rainfall and temperature
lag_days = 7
for i in range(1, lag_days + 1):
    auckland_rain[f'nextday_rainfall{i}'] = auckland_rain['Rainfall(mm)'].shift(i)
    auckland_rain[f'nextday_Deficit(mm){i}'] = auckland_rain['Deficit(mm)'].shift(i)
    auckland_rain[f'nextday_Tmax(C){i}'] = auckland_rain['Tmax(C)'].shift(i)
    auckland_rain[f'nextday_Tmin(C){i}'] = auckland_rain['Tmin(C) '].shift(i)
    auckland_rain[f'nextday_Tgmin(C){i}'] = auckland_rain['Tgmin(C)'].shift(i)
    auckland_rain[f'nextday_Tmean(C){i}'] = auckland_rain['Tmean(C)'].shift(i)

# Drop rows with NaN values due to lag features
auckland_rain.dropna(inplace=True)

auckland_rain.head(10)

KeyError: 'Tmin(C) '

In [ ]:
auckland_rain.columns

Index(['Date', 'Wdir (Deg)', 'WSpd (m/s)', 'GustDir (Deg)', 'GustSpd (m/s)',
       'Tdry (C)', 'Twet (C)', 'RH (%)', 'Tmax (C)', 'Tmin (C)', 'Tgmin (C)',
       'ET10 (C)', 'Pmsl (hPa)', 'Rad (MJ/m2)', 'Rain (mm)',
       'nextday_rainfall1', 'nextday_Tmax(C)1', 'nextday_Tmin(C)1',
       'nextday_Tmig(C)1', 'nextday_rainfall2', 'nextday_Tmax(C)2',
       'nextday_Tmin(C)2', 'nextday_Tmig(C)2', 'nextday_rainfall3',
       'nextday_Tmax(C)3', 'nextday_Tmin(C)3', 'nextday_Tmig(C)3',
       'nextday_rainfall4', 'nextday_Tmax(C)4', 'nextday_Tmin(C)4',
       'nextday_Tmig(C)4', 'nextday_rainfall5', 'nextday_Tmax(C)5',
       'nextday_Tmin(C)5', 'nextday_Tmig(C)5', 'nextday_rainfall6',
       'nextday_Tmax(C)6', 'nextday_Tmin(C)6', 'nextday_Tmig(C)6',
       'nextday_rainfall7', 'nextday_Tmax(C)7', 'nextday_Tmin(C)7',
       'nextday_Tmig(C)7'],
      dtype='object')

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into features and target variable
X = auckland_rain[['Wdir (Deg)', 'WSpd (m/s)', 'GustDir (Deg)', 'GustSpd (m/s)',
       'Tdry (C)', 'Twet (C)', 'RH (%)', 'Tmax (C)', 'Tmin (C)', 'Tgmin (C)',
       'ET10 (C)', 'Pmsl (hPa)', 'Rad (MJ/m2)', 'Rain (mm)',
        'nextday_Tmax(C)1', 'nextday_Tmin(C)1',
       'nextday_Tmig(C)1', 'nextday_rainfall2', 'nextday_Tmax(C)2',
       'nextday_Tmin(C)2', 'nextday_Tmig(C)2', 'nextday_rainfall3',
       'nextday_Tmax(C)3', 'nextday_Tmin(C)3', 'nextday_Tmig(C)3',
       'nextday_rainfall4', 'nextday_Tmax(C)4', 'nextday_Tmin(C)4',
       'nextday_Tmig(C)4', 'nextday_rainfall5', 'nextday_Tmax(C)5',
       'nextday_Tmin(C)5', 'nextday_Tmig(C)5', 'nextday_rainfall6',
       'nextday_Tmax(C)6', 'nextday_Tmin(C)6', 'nextday_Tmig(C)6',
       'nextday_rainfall7', 'nextday_Tmax(C)7', 'nextday_Tmin(C)7',
       'nextday_Tmig(C)7']]


# Define the target variable (next day's rainfall)
y = auckland_rain['nextday_rainfall1']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from xgboost import plot_importance

scaler = StandardScaler()

# Fit and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data using the same scaler
X_test_scaled = scaler.transform(X_test)


# Create and train the XGBoost model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.3, learning_rate=0.1,
                             max_depth=5, alpha=10, n_estimators=10, random_state=42)
xgb_model.fit(X_train_scaled, y_train)

# Plot feature importance
plot_importance(xgb_model)
plt.show()


# Train the model on the selected features
xgb_model.fit(X_train, y_train)

# Make predictions
xgb_predictions = xgb_model.predict(X_test)

# XGBoost Model

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt

# Make predictions on the training set
xgb_predictions_train = xgb_model.predict(X_train_scaled)

# Make predictions on the test set
xgb_predictions_test = xgb_model.predict(X_test_scaled)

# Calculate RMSE for training set
rmse_train = sqrt(mean_squared_error(y_train, xgb_predictions_train))

# Calculate MAE for training set
mae_train = mean_absolute_error(y_train, xgb_predictions_train)

# Calculate MSE for training set
mse_train = mean_squared_error(y_train, xgb_predictions_train)

# Calculate R2 score for training set
r2_train = r2_score(y_train, xgb_predictions_train)

# Print the metrics for the training set
print("XGBoost Training Set Metrics:")
print("Root Mean Squared Error (RMSE):", rmse_train)
print("Mean Absolute Error (MAE):", mae_train)
print("Mean Squared Error (MSE):", mse_train)
print("R2 Score:", r2_train)

# Calculate RMSE for test set
rmse_test = sqrt(mean_squared_error(y_test, xgb_predictions_test))

# Calculate MAE for test set
mae_test = mean_absolute_error(y_test, xgb_predictions_test)

# Calculate MSE for test set
mse_test = mean_squared_error(y_test, xgb_predictions_test)

# Calculate R2 score for test set
r2_test = r2_score(y_test, xgb_predictions_test)

# Print the metrics for the test set
print("\nXGBoost Test Set Metrics:")
print("Root Mean Squared Error (RMSE):", rmse_test)
print("Mean Absolute Error (MAE):", mae_test)
print("Mean Squared Error (MSE):", mse_test)
print("R2 Score:", r2_test)

c:\Users\Ying\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\Ying\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
c:\Users\Ying\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
c:\Users\Ying\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use

XGBoost RMSE: 6.3187544630682915


c:\Users\Ying\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\Ying\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
c:\Users\Ying\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
c:\Users\Ying\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use

In [ ]:
# Predict rainfall for tomorrow using the XGBoost model
xgb_predict_tomorrow = xgb_model.predict(X_train)
xgb_predict_day_after_tomorrow = xgb_model.predict(X_train)
xgb_predict_two_days_after_tomorrow = xgb_model.predict(X_train)

# Print the predictions
print(f'XgBoost Prediction for tomorrow: {xgb_predict_tomorrow[0]} mm')
print(f'XgBoost Prediction for the day after tomorrow: {xgb_predict_day_after_tomorrow[0]} mm')
print(f'XgBoost Prediction for two days after tomorrow: {xgb_predict_day_after_tomorrow[0]} mm')

Random Forest Prediction for tomorrow: -0.03590501844882965 mm
Random Forest Prediction for the day after tomorrow: -0.03590501844882965 mm
Random Forest Prediction for two days after tomorrow: -0.03590501844882965 mm


c:\Users\Ying\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\Ying\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
c:\Users\Ying\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
c:\Users\Ying\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use

In [ ]:
# Calculate monthly median rainfall
monthly_median_rainfall = auckland_rain.groupby(auckland_rain['Date(NZST)'].dt.month)['rainfall(mm)'].median()

# Calculate the percentage of actual rainfall compared to the monthly median
auckland_rain['rainfall_percentage'] = (auckland_rain['rainfall(mm)'] / monthly_median_rainfall[auckland_rain['date'].dt.month].values) * 100

# Create a function to categorize the weather conditions
def categorize_rainfall_condition(percentage):
    if percentage > 200:
        return "Very wet"
    elif percentage > 110:
        return "Moderately wet"
    elif percentage > 90:
        return "Near normal"
    elif percentage > 50:
        return "Moderately dry"
    else:
        return "Very dry"

# Apply the categorization function to create a new column with the weather condition
auckland_rain['weather_condition'] = auckland_rain['rainfall_percentage'].apply(categorize_rainfall_condition)

# Print the resulting dataframe with weather conditions
print(auckland_rain[['date', 'rainfall(mm)', 'rainfall_percentage', 'weather_condition']])


KeyError: 'Date(NZST)'